In [1]:
import os

In [5]:
os.getcwd()

'c:\\Data Engineering\\Jala\\vali_tools'

In [6]:
import pandas as pd

In [7]:
cyc = pd.read_csv(r"..\Source\Data\cycles.csv")

In [8]:
#data health
def check_health(df,dtype='all'):
    nulls = df.isnull().sum().reset_index()
    nulls.columns = ['col_name','null_count']
    unique = df.nunique().reset_index()
    unique.columns = ['col_name','unique_count']
    out = nulls.merge(unique,how='left',on='col_name')
    out['row_count'] = df.shape[0]
    out['null_pct'] = out['null_count']/out['row_count']
    out['unique_pct'] = out['unique_count'] / out['row_count']
    out['dupe_count'] = out['row_count'] - out['unique_count']
    out['dupe_pct'] = out['dupe_count']/out['row_count']
    try:
        dtype_int = df.dtypes.reset_index()
        dtype_int.columns = ['col_name','col_dtype']
        dtype_int.col_dtype = dtype_int.col_dtype.astype(str)
        int_col = dtype_int[dtype_int.col_dtype.isin(['int64','float64','datetime64[ns]'])]
        int_col = dtype_int[dtype_int.col_dtype.isin(['int64','float64','datetime64[ns]'])]
        int_info = df[int_col.col_name.values].describe().T.reset_index()
        int_info.columns = ['col_name',	'count','mean',	'min',	'25_pct','50_pct',	'75_pct',	'max','std']
        int_info = int_info[['col_name','mean','std','min','max','25_pct','50_pct','75_pct']]
        int_info['col_type'] = 'numeric'
            
        dtype_obj = df.dtypes.reset_index()
        dtype_obj.columns = ['col_name','col_dtype']
        dtype_obj.col_dtype = dtype_obj.col_dtype.astype(str)
        obj_col = dtype_obj[dtype_obj.col_dtype.isin(['object'])]
        obj_col = df[obj_col.col_name.values].describe().T.reset_index()
        obj_col.columns = ['col_name','count','unique','mode','freq']
        obj_col = obj_col[['col_name','mode','freq']]
        obj_col['col_type'] = 'object'
        out= out[['col_name','null_count','unique_count','row_count','dupe_count','null_pct','unique_pct','dupe_pct']]

        if dtype=='all':
            out_1 = out.merge(int_info,how='left',on='col_name').merge(obj_col,how='left',on='col_name')
            out_1['col_type_x'] = out_1['col_type_x'].fillna(out_1['col_type_y'])
            out_1.rename(columns={'col_type_x':'col_types'},inplace=True)
            return out_1.drop(columns='col_type_y')
        if dtype=='num_only':
            out_2 = out.merge(int_info,how='inner',on='col_name')
            return out_2
        if dtype=='obj_only':
            out_3 = out.merge(obj_col,how='inner',on='col_name')
            return out_3
        if dtype=='basic':
            return out
    except:
        return out


In [9]:
# cyc treatment
cyc.id = cyc.id.astype(str) 
cyc.pond_id = cyc.pond_id.astype(str) 
cyc.species_id = cyc.species_id.astype(str)
cyc = cyc.drop_duplicates(subset='id')
cyc.started_at = pd.to_datetime(cyc.started_at)
cyc.finished_at = pd.to_datetime(cyc.finished_at)
cyc.created_at = pd.to_datetime(cyc.created_at)
cyc.updated_at = pd.to_datetime(cyc.updated_at)
cyc.extracted_at = pd.to_datetime(cyc.extracted_at)
cyc.ordered_at = pd.to_datetime(cyc.ordered_at)

In [10]:
check_health(cyc)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,id,0,2500,2500,0,0.0000,1.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,20849,1
1,pond_id,0,1675,2500,825,0.0000,0.6700,0.3300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,24569,8
2,species_id,0,3,2500,2497,0.0000,0.0012,0.9988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,1.0,1576
3,total_seed,0,868,2500,1632,0.0000,0.3472,0.6528,217154.5784,170662.188169,10.0,1800000.0,90000.0,184722.0,300000.0,numeric,NaN,NaN
4,started_at,0,750,2500,1750,0.0000,0.3000,0.7000,2022-10-27 02:43:00.479999744,NaN,2020-03-07 00:00:00,2024-02-18 00:00:00,2022-05-08 00:00:00,2022-11-16 00:00:00,2023-08-05 00:00:00,numeric,NaN,NaN
5,finished_at,1,857,2500,1643,0.0004,0.3428,0.6572,2023-01-13 17:38:32.124849920,NaN,2020-05-20 00:00:00,2024-04-02 00:00:00,2022-07-16 12:00:00,2023-02-10 00:00:00,2023-10-26 00:00:00,numeric,NaN,NaN
6,remark,1172,67,2500,2433,0.4688,0.0268,0.9732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,#SiklusFarm,269
7,created_at,0,2325,2500,175,0.0000,0.9300,0.0700,2022-11-10 21:50:26.408800,NaN,2020-02-19 08:44:53,2024-03-05 13:10:13,2022-05-17 04:53:08,2022-12-02 04:03:08,2023-08-10 03:05:55.750000128,numeric,NaN,NaN
8,updated_at,0,2289,2500,211,0.0000,0.9156,0.0844,2023-03-08 00:19:24.160000,NaN,2020-06-03 02:44:52,2024-04-09 00:06:26,2022-08-25 12:30:48.500000,2023-06-05 00:03:12,2023-11-20 07:32:10,numeric,NaN,NaN
9,area,0,648,2500,1852,0.0000,0.2592,0.7408,2233.092964,12372.846032,1.02,422500.0,803.84,1368.0,2238.6825,numeric,NaN,NaN
